### 將原始評論過濾掉外文,emoji,過多的英文並轉換出各分類次評等

In [2]:
import pandas as pd
import numpy as np
import re
import emoji
from collections import Counter
import warnings
from datetime import datetime
warnings.filterwarnings("ignore")
from opencc import OpenCC
import seaborn as sns
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER

In [ ]:
def is_japenese(uchar):
    if (uchar >= u'\u30a0' and uchar <= u'\u30ff') or (uchar >= u'\u3040' and uchar <= u'\u309f'):
        return True
    else:
        return False
def checkjapenese(sen):
    e=0
    c=0
    for i in sen:
        if is_japenese(i):
            e+=1
        if is_chinese(i):
            c+=1
    if c>0:
        return(e/c>0.5)
    else:
        return(True)
def is_alphabet(uchar):
    if (uchar >= u'\u0041' and uchar <= u'\u005a') or (uchar >= u'\u0061' and uchar <= u'\u007a'):
        return True
    else:
        return False
def is_chinese(uchar):
    if uchar >= u'\u4e00' and uchar <= u'\u9fa5':
        return True
    else:
        return False
def checkEng(sen):
    e=0
    c=0
    for i in sen:
        if is_alphabet(i):
            e+=1
        if is_chinese(i):
            c+=1
    if c>0:
        return(e/c>0.5)
    else:
        return(True)


cc = OpenCC('s2t')
allLang=pd.read_csv('./crawler/TPE_GCP_allLang.csv',low_memory=False)
allLang=allLang[(allLang['review'].apply(checkEng)==False)&(allLang.auto_translate==False)&(allLang['review'].apply(checkjapenese)==False)]

allLang.total_dict=allLang.total_dict.apply(lambda x:eval(x))
ch_list=[]
for list_ in allLang['total_dict'].apply(lambda x:x.keys()):
    for key in list_:
        if key not in ch_list:
            ch_list.append(key)
ch_list=ch_list[:-4]
en_list=['cp','place','service','comfort','clean','sleep']
for en,ch in zip(en_list,ch_list):
    allLang[f'is_{en}']=allLang['total_dict'].apply(lambda x:x.get(ch))
    
allLang=allLang.reset_index().rename(columns={'index':'old_index'})
allLang.to_csv('original_review.csv',index=False)

### 情緒詞根據出現次數進行過濾後，並進行人工識別建立旅遊評論情感詞字典

In [ ]:
sym_list=["“","”","（","）","(",")","【","】","：",":",";",'"',"'","/","-","～","~","⋯",".","=",'*',"^","#",'_','▽','´',"`"]
def extract_emojis(s): ##簡繁轉換,情緒符號
    s=cc.convert(s)
    s=s.replace('ㄧ','一')
    for sym in sym_list:
        s=s.replace(sym," ")
    return ''.join(c for c in s if c not in emoji.UNICODE_EMOJI)
## 進行斷詞
ws = WS("./outsourced_data")
pos = POS("./outsourced_data")
original_review=pd.read_csv('original_review.csv')
original_review=original_review[['old_index','review']]
original_review.columns=['old_index','ori_review']
original_review['ori_review']=original_review['ori_review'].apply(extract_emojis)
original_review['token_ckip']=original_review['ori_review'].apply(lambda x:ws([x]))
original_review['pos_ckip']=original_review['token_ckip'].apply(pos)
original_review=original_review.reset_index(drop=True)
def merge_pos(x):
    pos_list=[]
    for token ,pos in zip(x['token_ckip'][0],x['pos_ckip'][0]):
        if pos=='WHITESPACE':
            continue
        pos_list.append((token,pos))
    return(pos_list)
original_review['merge_pos']=original_review.apply(merge_pos,axis=1)
original_review.to_csv('token_pos_ckip.csv',index=False)

### 根據中斷標點符號進行切割句子

In [ ]:
token_pos_ckip=pd.read_csv('token_pos_ckip.csv')##old_index,sentence,pos
token_pos_ckip['merge_pos']=token_pos_ckip['merge_pos'].apply(eval)
total_sentence=[]
for old_index,merge_pos in zip(token_pos_ckip['old_index'],token_pos_ckip['merge_pos']):
    wordString=''
    sentence_list=[]
    for i,word_pos in enumerate(merge_pos):
        word,pos=word_pos
        wordString+=word
        sentence_list.append(word_pos)
        if( pos in ['PERIODCATEGORY','ETCCATEGORY','EXCLAMATIONCATEGORY','QUESTIONCATEGORY','SEMICOLONCATEGORY']):
            total_sentence.append([old_index,wordString[:-1],sentence_list[:-1]])
            wordString=''
    total_sentence.append([old_index,wordString,sentence_list])

cut_sentence=pd.DataFrame(total_sentence,columns=['old_index','sentence','pos'])

cut_sentence=cut_sentence[cut_sentence.sentence!='']

cut_sentence.to_csv('By_sentence_pos.csv',index=False)